# Agent
Agent를 활용하여 Sql 쿼리를 생성하고 실행 결과를 답변으로 출력이 가능합니다.

- `langchain_community.agent_toolkits` 안에있는 `create_sql_agent` 함수를 이용합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

In [4]:
# 데이터베이스의 방언(dialect)을 출력합니다.
print(db.dialect)

sqlite


In [5]:
# 사용 가능한 테이블 이름들을 출력합니다.
print(db.get_usable_table_names())

['accounts', 'customers', 'transactions']


In [6]:
# Agent 생성
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [7]:
# 실행 결과 확인
agent_executor.invoke(
    {"input": "테디와 셜리의 transaction 의 합계를 구하고 비교하세요"}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(account_id) REFERENCES accounts (account_id)
)

/*
3 rows from transactions table:
transaction_id	account_id	amount	transaction_date
1	1	74.79	2024-07-13
2	1	-224.1	2024-05-13
3	1	-128.9	2024-01-25
*/
Invoking: `sql_db_schema` with `{'table_names': 'customers'}`



CREATE TABLE customers (
	customer_id INTEGER, 
	name TEXT, 
	age INTEGER, 
	email TEXT, 
	PRIMARY KEY (customer_id)
)

/*
3 rows from customers table:
customer_id	name	age	email
1	테디	30	teddy@example.com
2	폴	40	paul@example.com
3	셜리	25	shirley@example.com
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT c.name, SUM(t.amount) AS total_amount\nFROM transactions t\nJOIN 

{'input': '테디와 셜리의 transaction 의 합계를 구하고 비교하세요',
 'output': 'The total transaction amounts for Teddy and Shirley are as follows:\n\n- **Teddy**: -965.7\n- **Shirley**: 656.64\n\nThis indicates that Teddy has a negative total, suggesting he has more withdrawals than deposits, while Shirley has a positive total.'}

In [8]:
agent_executor.invoke(
    {"input": "테디와 셜리의 transaction 의 합계를 구하고 비교하세요"}
)["output"]



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY(account_id) REFERENCES accounts (account_id)
)

/*
3 rows from transactions table:
transaction_id	account_id	amount	transaction_date
1	1	74.79	2024-07-13
2	1	-224.1	2024-05-13
3	1	-128.9	2024-01-25
*/
Invoking: `sql_db_schema` with `{'table_names': 'customers'}`



CREATE TABLE customers (
	customer_id INTEGER, 
	name TEXT, 
	age INTEGER, 
	email TEXT, 
	PRIMARY KEY (customer_id)
)

/*
3 rows from customers table:
customer_id	name	age	email
1	테디	30	teddy@example.com
2	폴	40	paul@example.com
3	셜리	25	shirley@example.com
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT SUM(amount) AS total_amount, c.name FROM transactions t JOIN acco

'테디의 transaction 합계는 -965.7입니다. 셜리의 transaction 데이터는 없어서 비교할 수 없습니다.'